# Topic Modeling and Latent Dirichlet Allocation (LDA) in Python

In [2]:
import nltk
import numpy as np
from nltk.stem.porter import *
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import gensim
import pandas as pd

In [3]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

Length of documents

In [5]:
len(documents)

1103663

First documents

In [7]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Data Pre-Processing
Performing the following steps:

- Tokenisation: Split the words into sentences and the sentences into words. Lowercase the words and remove punctuation
- Words that have fewer than 3 characters are removed
- All stopwords are removed
- Words are lemmatized: words in third person are changed to first person and verbs in past and future tenses are changed into present
- Words are stemmed: words are reduced to their root form

In [9]:
np.random.seed(2018)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/farhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [11]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

## Select a document to preview after preprocessing

In [12]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenised and lemmatised document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenised and lemmatised document: 
['rain', 'help', 'dampen', 'bushfir']


## Preprocess the headline text, saving the result as 'processed_docs'

In [13]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Bag of Words on the Data set

Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set.

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [15]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


## Gensim filter_extremes
Filter out tokens that appear in:

- less than 15 documents (absolute number) or
- more than 0.5 documents (fraction of total corpus size, not absolute number)
- after the above two steps, keep only the first 100000 most frequent tokens

In [16]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## Gensim doc2bow
For each document we create a dictionary reporting how many words and how many times these words appear. Save this to 'bow_corpus', then check our selected document earlier

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [19]:
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4022, 1)]

In [22]:
len(bow_corpus)

1103663

## Preview Bag of words for our sample preprocessed document

In [20]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],
                                                    dictionary[bow_doc_4310[i][0]],
                                                    bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4022 ("dampen") appears 1 time.


## TF-IDF
Create tf-dif model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf', then apply transformation to the entire corpus and call it 'corpus_tfidf'. Finally we preview TF-IDF scores for our first document.

In [21]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903873663261261),
 (1, 0.38522586675616677),
 (2, 0.49747840165121515),
 (3, 0.5055284652272439)]


## Running LDA using Bag of Words
Train our LDA model using gensim.models.LdaMulticore and save it to 'lda_model'

In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"adelaid" + 0.018*"market" + 0.016*"south" + 0.013*"share" + 0.012*"farmer" + 0.011*"victoria" + 0.011*"busi" + 0.011*"flood" + 0.010*"water" + 0.010*"farm"
Topic: 1 
Words: 0.056*"australia" + 0.036*"trump" + 0.026*"world" + 0.022*"nation" + 0.015*"tasmania" + 0.013*"island" + 0.011*"polit" + 0.010*"hobart" + 0.010*"take" + 0.009*"drum"
Topic: 2 
Words: 0.024*"plan" + 0.018*"council" + 0.014*"health" + 0.014*"indigen" + 0.013*"turnbul" + 0.013*"chang" + 0.013*"commun" + 0.011*"meet" + 0.011*"park" + 0.011*"servic"
Topic: 3 
Words: 0.035*"australian" + 0.025*"kill" + 0.021*"attack" + 0.019*"famili" + 0.017*"donald" + 0.015*"fight" + 0.013*"dead" + 0.012*"crash" + 0.011*"victim" + 0.011*"million"
Topic: 4 
Words: 0.020*"test" + 0.020*"melbourn" + 0.020*"countri" + 0.018*"live" + 0.017*"coast" + 0.014*"gold" + 0.014*"women" + 0.013*"life" + 0.013*"feder" + 0.013*"student"
Topic: 5 
Words: 0.029*"elect" + 0.022*"canberra" + 0.018*"rural" + 0.016*"say" + 0.013*"worke